# How to detect subcellular Ca<sup>2+</sup> signals using Flika 
by Kyle Ellefsen, January 24, 2016


This tutorial is relevant for researchers who record subcellular Ca<sup>2+</sup> signals using fluorescent Ca<sup>2+</sup> indicators.  This algorithm automates the detection and analysis of these brief Ca2+ signals.  This is not a tutorial for researchers looking to analyze global Ca<sup>2+</sup> signals, although I'm planning on creating a tutorial for that, and many of the same concepts apply.

In this problem, we will use the threshold-cluster algorithm.  This algorithm is not published, but is modified from the algorithm published in [Ellefsen et al, 2014](http://dx.doi.org/10.1016/j.ceca.2014.06.003).  When using this algorithm, please cite that paper.  The threshold-cluster algorithm can be conceptually divided into two steps.  The first step preprocesses the raw movie and creates three movies.  The second step performs analysis on those movies according to user defined parameters.  I'll break this tutorial into those two steps.  

Using this algorithm involves reading and writing code.  Although this algorithm can all be done using Flika's GUI tools (opening windows, clicking buttons, etc.), it is much more efficient for a researcher to write a python file that will automatically perform all these operations.  Therefore, in this tutorial, I will be describing the code you'll need to use in each step.  If you do not know how to code in python, I recommend downloading [anaconda](https://www.continuum.io/downloads), which contains python and many python packages.  Then launch [Spyder](https://github.com/spyder-ide/spyder) (which comes with anaconda), and from within Spyder, run [Flika.py](https://github.com/kyleellefsen/Flika).  All these commands will be executed in the same console that Flika.py is running in. If you have questions about how to do that, email me and I'll help you with setting up.

![step flowchart](images/flowchart.png)

## 1. Preprocessing
There are three movies we want to generate as part of the preprocessing step.
1. $F/F_0$ Movie
2. Normalized Movie
3. Binary Movie

First, let's generate the $F/F_0$ movie. Each value of the $F/F_0$ movie represents a percent change from baseline fluoresences.  A value of 1 indicates that pixel is currently exactly at baseline fluoresence.  A value of 1.14 indicates a 14% increase above baseline fluorsence. To generate this movie, we will first open our file.

In [ ]:
open_file()

Then we will subtract the camera's black level.  The black level is the value of a pixel when 0 photons are detected.  There are two common ways to measure black level.  You could turn the laser off at the beginning of your record, and switch it on when you've recorded enough frames to accurately determine the black level.  Or you could measure the values in a black area of your field of view.  Let's say our black level is 120.

In [ ]:
subtract(120)

Operations in Flika are performed on the current window, which is either the window that was most recently created or the window that was last clicked on.  

Now, for each pixel, we want the percent change from baseline.  To calculate this, we need a measure of baseline fluorescence, during which there is no Ca<sup>2+</sup> activity.  Let's suppose that our cell is quiescent from frame 0 to frame 30.  Then we will take the average image from frame 0 to 30, and divide every image in the movie by that average image.  In Flika, this is performed in a single command:

In [ ]:
data_window=ratio(150,300,'average'); #ratio(first_frame, nFrames, ratio_type), now we are in F/F0
data_window.setName('Data Window dF/F0')

Notice that we set the variable 'data_window'.  The output of each Flika command returns the new window created by the command, which we can assign to a variable for later use.  We now have our data_window.  

The normalized movie should have several special properties that will enable automated signal detection.  The noise for each pixel should be gaussian distributed with a mean of 0 and a standard deviation of 1 ($\mu = 0; \sigma =1$).  Since data files won't start off with those properties, we need to transform them. 

Since we are concerned with only local signals, sometimes we want to remove global signals or other slow noise.  We can exploit the fact that global signals typically have a much slower timecourse than brief local signals, and use a temporal filter to remove slow changes.  We will use the butterworth filter.  In order to determine what parameters we want to use, let's use the the butterworth filter GUI.  Right click and drag on the $F/F_0$ movie window to create an ROI.  Right click in the center of that ROI, and plot the trace.  Then, in the main Flika window, select Process->Filters->Butterworth Filter.  Make sure the Preview checkbox is checked.  Adjust the Low Cutoff Frequency until the global signals are removed, but the local signals are still visible.  Then press the 'Cancel' button and use the parameters you found to perform the filter.  Sometimes this filter can take a few minutes.



In [ ]:
butterworth_filter(1,.03,1,keepSourceWindow=True)

If you don't need to filter your data, just subtract 1 from each value so that the noise is centered around 0.  

In [ ]:
subtract(1) #Don't do this if you used the filter above

Now our movie should have its noise centered around 0.  We want the standard deviation of our noise to be equal to 1, so we'll find a quiescent section of our movie and ratio by the standard deviation, similar to when we ratioed by the average.

In [ ]:
ratio(100,100,'standard deviation')  #ratio(first_frame, nFrames, ratio_type), now the standard devation of the noise is 1.

Our butterworth filter created artifacts at the beginning and end of the movie.  Let's eliminate those by setting the values at the beginning and end of the movie to 0. 'mt' is a variable that represents number of frames of our movie.

In [ ]:
norm_window=set_value(0,0,100) #to get rid of butterworth artifact at the beginning of the movie
norm_window=set_value(0,mt-150,mt-1) #to get rid of butterworth artifact at the end of the movie
norm_window.setName('Normalized Window')

Great! We're most of the way there.  We have our $F/F_0$ Movie and our Normalized Movie.  The last movie we need is the Binary Movie.  If we threshold our normalized window, so that all the pixels below a certain value are set to 0 and all pixels above that value are set to 1, the result is a binary movie.  Picking the value of the threshold is about balance.  If the threshold is too high, you will miss detecting small events.  If the threshold is too low, then calculating the density (described below) will take a very long time.  I've found in practice that a value of around 1 works well, but this will vary depending on your movie.

In [ ]:
binary_window=threshold(1)

## 2. Clustering
The second half of the algorithm requires only those three movies we created earlier:  
1. $F/F_0$ Movie
2. Normalized Movie
3. Binary Movie



We pass these three movies into the 'threshold_cluster()' function, along with some constants, and the algorithm detects subcellular signals! And like many black boxes, for it to work correctly, we have to know what it's doing under the hood.  So what does this function do?

### Density Calculation
First it uses the binary movie to calculate density.  The logic behind this is as follows: Where there is no signal, the pixels in the binary movie which are true should be randomly, uniformly spread.  If you pick any point anywhere in the image and draw a sphere around that point (like a circle in the x and y dimensions, but also extended into the time dimension), and count the number of true pixels inside that sphere, the number of pixels will be distributed according to the binomial distribution. The specifics of the math doesn't really matter, what's important is that this number (the number of true pixels inside a sphere without a signal) will be different than the number of true pixels inside a sphere when there is a signal.  In the presence of a signal, the number of true pixels will be higher.  

To get the density, we calculate, for each true pixel, the number of true pixels in a sphere around it.  We then store that value in another movie at the same location.  The result looks like this:

![binary to density](images/binary_to_density.PNG)

You can see that the pixels that occur at high density stand out. After the density transformation they have a much greater value relative to the background true pixels than in the $F/F_0$ Movie.  The density window wiill ask you for a density threshold (which will be prefilled with the argument 'density_threshold' you passed to the threshold_cluster() function). All pixels below this threshold will not be included in clusters, everything above will be assigned a cluster.  You can adjust the lookup table (the blue area on the right of the window) to see what different thresholds will look like.  Once you have entered a threshold, press the 'Set Threshold' button at the bottom of the window to move to the next step. 

### Clustering
The clustering algorithm is based on the one published by [Rodriguez and Laio, 2014](http://doi.org/10.1126/science.1242072). The basic idea is simple.  We have a bunch of pixels with positive values from our density transformation.  For each pixel, we find nearest pixel with a higher value.  For pixels that are on the edge of an event, those pixels with higher values will be close.  See the little cartoon I made below.  The green arrows show pixels pointing to the pixel identified as having a higher value.
![clustering cartoon](images/clustering_cartoon.PNG)
One arrow stands out.  One pixel near the center of the cluster has the highest value, so the arrow from it is much much longer than any of the other arrows in the cluster. In general, all clusers should have this property: one pixel near the center should be pointing to a pixel that is pointing far away, every other pixel in the cluster will have small arrows.  From this principle, we can find clusters.  All we need to do is identify the pixels that have abnormally high distances to the closest pixel of higher value.  These pixels should also have an abnormally high density, since many isolated pixels will also have abnormally high arrows.  I've simulated a few signals and plotted the density of each true pixel vs the smallest distance to a denser point.

![Density vs Distance](images/density_vs_distance.PNG)

You can see that there are several points that have a high density and distance.  These are the centers of the clusters.  The clustering algorithm considers all pixels that point to these centers to be part of these clusters.  Circle the points at the top right of the graph, and Flika will output a movie where each cluster is colored differently.  This way you can be sure that you've clustered how you would like.  

![Density vs Distance (circled)](images/density_vs_distance_circled.PNG)

The pixels that are considered 'centers' will be colored green after circling.  Red indicates that there were fewer than 10 pixels in the cluster, so that cluster was automatically rejected.  

![clusters](images/clusters.PNG)

If you don't like the way that you clustered, you can recircle the pixels to recluster.  Once you've got them clustered the way you like, click back on density vs distance window and press enter.  This will bring you to the Puff Analyzer dialog.

## Puff Analyzer
If you look back at the $F/F_0$ window, it will have a bunch of red dots on it.  Each one of those dots represents a detected Ca2+ signal.

![Map of Event Centroids](images/puff_map.png)

The locations of the dots were determined by fitting a 2D gaussian to the event.  A square region around the event is cut out, and all the frames between the start frame and end frame of the event are averaged, pixel-by-pixel.  This averaged image is then fit, and the centroid of that event is plotted as a red dot.  The figure below shows an averaged image and the resulting gaussian fit.  The accuracy of that centroid, assuming the underlying event is stationary and gaussian shaped, is described [here](http://scitru.com/articles/3/). 

![Gaussian Fitting](images/gaussian_fitting.PNG)

If you click on the red outline of the event in the trace window, or if you click on the centroid in the $F/F_0$ window, the Puff Analyzer dialog will display the trace for that signal.  The trace is created by averaging the values of the pixels in a square centered around the centroid of the gaussian fit.  The width of the square is specified by the user (the parameter 'roi_width').  The yellow vertical lines denote the frames of the first and last pixel in the cluster.  The lower horizontal line is the amplitude of the trace at the first frame.  The horizontal line is the maximum amplitude of the trace.  The vertical lines can be adjusted by clicking and dragging.  Once adjusted, all the kinetics are recalculated for that event.  All these values are in terms of $F/F_0$, and all of them are saved in the excel spreadsheet when you press the button "Export to Excel".

![Event Trace](images/puff_trace.PNG)

Here are a few of the things you can do to explore your data:

* Right click and drag inside the ROI to move it around.  Drag near a corner to change its size.

* Delete all events in the ROI by pressing the delete button when your mouse is inside the ROI.

* Delete a single puff by pressing the delete button when that puff is selected and your mouse is outside the ROI.

* Group all the events in the ROI by pressing the 'g' button.

* Ungroup by pressing the 'u' button.

* To link views between the data window and any other window (such as the normalized window), right click on the ROI and select 'copy'. Then right click on the normalized window and select 'paste'. Then right click on the new ROI and select 'plot'. A line will appear that will plot the average inside that ROI for the normalized window.

* Press the 'toggle groups' button to view the green group dots.  Click on one of these dots to bring up the group analyzer window.

### Export to Excel
When the "Export to Excel" button is pressed, three sheets are made.  
#### Event Data
The "Event Data" sheet contains the information about each event.  The events are grouped into "Groups", each with a group number.  These groupings are determined by proximity of event centroids to one another.  If centroids are within the radius set by the argument 'radius', the events are grouped together.  

Each row of the sheet represents one event.  Included in this data are the rise and fall times.  r20 is the number of frames the between the first frame of the event and the frame where the event's trace exceeded 20% of the maximum value.  r100 is the number of frames of the entire rising phase.  f80 is "fall to 80%" time.  etc.
#### Group traces
Each column of group traces represents one group, and the rows represent the average value of a square centered around the (unweighted) mean of the centroids of the events making up that group.  The width of the square is determined by the 'roi_width' argument.  
#### Peak aligned Event Traces
These traces are the same traces as displayed in the "Puff Analyzer" window.  They are aligned to their peaks.  

### Reloading Data into Flika
Most researchers will want to load data that they have already analyzed. To do that, in the Puff Analyzer control panel, press the Save (.flika) button.  This will save all the information about the events (except the 3 movies created in the preprocessing step) in a .flika file which will be located in the same location and with the same basename as the original image file.  

The next time you run the threshold_cluster() function using a data_window that was created from the file you already analyzed, if the 'load_flika_file' argument is set to True (it is by default), it will load that flika file and all the events will be exactly as you left them last time.  For example, the events that you trashed before will still be trashed.  You also don't need the binary_window or the highpass_window arguments when loading the flika file, so you can pass None to these arguments.  Here is an example script which will load data I've previously analyzed.



In [ ]:
from plugins.detect_puffs.threshold_cluster import threshold_cluster
filename=r'C:/Users/Kyle Ellefsen/Desktop/tmp.tif'
open_file(filename)
subtract(10) #subtract camera black level 
data_window=ratio(150,300,'average'); #ratio(first_frame, nFrames, ratio_type), now we are in F/F0
data_window.setName('Data Window dF/F0')
threshold_cluster(None,data_window,None)